### パッケージのインポート

In [ ]:
import openpyxl
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Border, Side

### データフレーム作成

In [ ]:
df_project = pd.read_excel("../プロジェクト実績集計（社員・日々）.xlsx", usecols=range(7))
df_sum1 = pd.read_excel("../月次合計一覧 (1).xlsx", usecols=range(7))
df_sum2 = pd.read_excel("../月次合計一覧.xlsx", usecols=range(21))

#### チェック用ダミーデータ作成と追加

In [ ]:
# ダミーデータの追加
dummy_data = {
    "就労所属コード": ["L00X"],
    "就労所属名称": ["テスト課"],
    "社員番号": ["T0001"],
    "社員名称": ["テスト　太郎"],
    "総労働時間": ["190:00"],
    "月の法定限度時間": ["177:00"],
    "安衛法超過時間": ["13:00"],
}

df_dummy = pd.DataFrame(dummy_data)

df_sum1 = df_sum1.append(df_dummy, ignore_index=True)

# ダミーデータの追加
dummy_data2 = {
    "就労所属コード": ["L00X"],
    "就労所属名称": ["テスト課"],
    "社員番号": ["T0002"],
    "社員名称": ["テスト　花子"],
    "総労働時間": ["240:00"],
    "月の法定限度時間": ["177:00"],
    "安衛法超過時間": ["63:00"],
}

df_dummy = pd.DataFrame(dummy_data2)

df_sum1 = df_sum1.append(df_dummy, ignore_index=True)

dummy_data3 = {
    "就労所属コード": ["L00X"],
    "就労所属名称": ["テスト課"],
    "社員番号": ["T0002"],
    "社員名称": ["テスト　花子"],
    "所定労働時間": ["240:00"],
    "フレックス\n総労働時間": ["対象外"],
    "残業時間": ["63:00"],
    "フレックス\n超過時間": ["対象外"],
    "所定休日出勤時間": ["0:00"],
    "法定休日出勤時間": ["0:00"],
    "深夜時間": ["0:00"],
    "減時間": ["0:00"],
    "フレックス\n不足時間": ["0:00"],
    "遅刻回数": ["0"],
    "早退回数": ["0"],
    "私用外出回数": ["0"],
    "宿直回数": ["0"],
    "出向回数": ["0"],
    "夜勤回数": ["0"],
    "テレワーク回数": ["0"],
    "不就業時間": ["0:00"],
}

df_dummy3 = pd.DataFrame(dummy_data3)
df_sum2 = df_sum2.append(df_dummy3, ignore_index=True)

dummy_data4 = {
    "就労所属コード": ["L00X"],
    "就労所属名称": ["テスト課"],
    "社員番号": ["T0002"],
    "社員名称": ["テスト　太郎"],
    "所定労働時間": ["対象外"],
    "フレックス\n総労働時間": ["190:00"],
    "残業時間": ["対象外"],
    "フレックス\n超過時間": ["13:00"],
    "所定休日出勤時間": ["0:00"],
    "法定休日出勤時間": ["0:00"],
    "深夜時間": ["0:00"],
    "減時間": ["0:00"],
    "フレックス\n不足時間": ["0:00"],
    "遅刻回数": ["0"],
    "早退回数": ["0"],
    "私用外出回数": ["0"],
    "宿直回数": ["0"],
    "出向回数": ["0"],
    "夜勤回数": ["0"],
    "テレワーク回数": ["0"],
    "不就業時間": ["0:00"],
}

df_dummy4 = pd.DataFrame(dummy_data4)
df_sum2 = df_sum2.append(df_dummy4, ignore_index=True)

#### 時間を変換するための関数

In [ ]:
# 時間に変換するための関数
def time_to_minutes(time):
    h, m = map(int, time.split(":"))
    return h * 60 + m


# 分を時間に変換するための関数
def minutes_to_time(minutes):
    h = minutes // 60
    m = minutes % 60
    return f"{h}:{m:02d}"

### 超過時間チェック関数

引数は読み込んだデータフレーム 

In [ ]:
# 月次合計一覧から労働時間の超過を算出
def overworkcheck(df):
    # 超過している行を抽出
    overtime_df = df[df["安衛法超過時間"].apply(time_to_minutes) > 0]

    # 必要な列を表示
    return overtime_df[
        ["就労所属コード", "就労所属名称", "社員番号", "社員名称", "総労働時間", "月の法定限度時間", "安衛法超過時間"]
    ]

In [ ]:
# 関数の実行
overtime_df = overworkcheck(df_sum1)

In [ ]:
overtime_df

### 時間超過している個人の情報をExcelに書き込む

#### 使用する変数、パスまとめ

In [ ]:
department_name = "開発管理部"  # 対象の部署名
input_filename = "../ref/残業申請サンプル.xlsx"  # 読み込むファイル名
output_filename = "../result/result.xlsx"  # 書き込むファイル名

# 部署名と対象の月
month = 6

In [ ]:
# df_sum1とdf_sum2で、”社員名称”が共通している行のデータを取り出す関数
def get_overtime_hours(df_sum1, df_sum2):
    # 社員名称をキーにしてdf_sum2から残業時間またはフレックス超過時間を取得
    def find_overtime(row):
        employee_name = row["社員名称"]
        matched_row = df_sum2[df_sum2["社員名称"] == employee_name]
        if matched_row.empty:
            return ""
        overtime = matched_row.iloc[0]["残業時間"]
        flex_overtime = matched_row.iloc[0]["フレックス\n超過時間"]
        if overtime != "対象外":
            hours, _ = map(int, overtime.split(":"))
            return f"{hours}h"
        elif flex_overtime != "対象外":
            hours, _ = map(int, flex_overtime.split(":"))
            return f"{hours}h"
        return ""

    df_sum1["今月の残業時間"] = df_sum1.apply(find_overtime, axis=1)

    return df_sum1


def write_overtime_to_excel(
    df, department_name, input_file_name, output_filename, start_cell="C6"
):
    # Excelファイル読み込み
    wb = load_workbook(input_filename)
    ws = wb.active

    # 引数のセル番号を行番号と列番号に変換
    start_row, start_col = openpyxl.utils.cell.coordinate_to_tuple(start_cell)

    count = 0  # カウントの初期化

    # データを書き込む
    for row_idx, (_, row) in enumerate(df.iterrows(), start=start_row):
        ws.cell(row=row_idx, column=start_col, value=row["社員名称"])
        ws.cell(row=row_idx, column=start_col + 1, value=department_name)
        ws.cell(row=row_idx, column=start_col + 2, value=row["就労所属名称"])
        ws.cell(row=row_idx, column=start_col + 3, value=row["今月の残業時間"])

        # Merge cells for "主な業務内容" in the data rows
        ws.merge_cells(
            start_row=row_idx,
            start_column=start_col + 6,
            end_row=row_idx,
            end_column=start_col + 7,
        )
        if count > 0:
            count += 1  # 2つ目以降の場合、カウントをインクリメント

        if count == 0:
            count += 1  # 初回処理後にカウントを1に設定

    # 罫線で表を作る
    rows = len(df)  # 行の数
    cols = 7  # 列の数
    thin_border = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin"),
    )

    for row in ws.iter_rows(
        min_row=start_row,
        max_row=start_row + rows - 1,
        min_col=start_col,
        max_col=start_col + cols - 1,
    ):
        for cell in row:
            cell.border = thin_border

    # ファイルを保存
    wb.save(output_filename)

In [ ]:
# 個人の残業時間を抽出し、Excelに個人の情報を書き込み
overtime_df = get_overtime_hours(overtime_df, df_sum2)
write_overtime_to_excel(overtime_df, department_name, input_filename, output_filename)

### 月次合計一覧から、フレックスを含む残業時間を合算する関数

月次合計一覧のデータフレーム df_sum2 を使用

In [ ]:
# 残業時間とフレックス超過時間の列を変換し、合計を合算
def convert_to_minutes(time):
    if "対象外" in time or not time:
        return 0

    # "HH:MM:SS" 形式も "HH:MM"形式も考慮
    parts = time.split(":")
    if len(parts) == 3:
        h, m, s = map(int, parts)
    elif len(parts) == 2:
        h, m = map(int, parts)
        s = 0
    else:
        return 0

    return h * 60 + m


# 合計残業時間を計算
def calculate_total_time(df, columns):
    total_minutes = sum(df[col].apply(convert_to_minutes).sum() for col in columns)

    return minutes_to_time(total_minutes)

In [ ]:
# 合算する対象の列
columns_to_sum = ["残業時間", "フレックス\n超過時間"]

# 時間を算出する
total_time = calculate_total_time(df_sum2, columns_to_sum)

# 結果の出力
print(total_time)

### 算出したデータをExcelに出力

### 部署内の総残業時間書き込み自動化

引数は出力ファイル、部署名、月、総残業時間

書き込み保存するところまで関数内で完結させる

In [ ]:
def update_overtime_request(filepath, department, month, data):
    # 部署名と対応する行番号の辞書
    start_col = 19
    department_row_mapping = {
        "本部長付": start_col,
        "開発管理部": start_col + 3,
        "開発推進部": start_col + 6,
        "1開発設計部": start_col + 9,
        "ディバイス開発部": start_col + 12,
        "電気・電子開発部": start_col + 15,
        "開発本部": start_col + 18,
    }
    # 月と対応する列番号の辞書
    month_column_mapping = {
        1: "E",
        2: "F",
        3: "G",
        4: "H",
        5: "I",
        6: "J",
        7: "K",
        8: "L",
        9: "M",
        10: "N",
        11: "O",
        12: "P",
    }

    # 対象の行番号と列番号を取得
    row = department_row_mapping.get(department)
    column = month_column_mapping.get(month)

    if row is None:
        raise ValueError(f"指定された部署名'{department}'が無効です。")
    if column is None:
        raise ValueError(f"指定された月'{month}'が無効です。")

    # Excelファイル読み込み
    wb = load_workbook(filepath)
    ws = wb.active

    # セルにデータを入力
    cell = f"{column}{row}"
    ws[cell] = data

    # ファイルを保存
    wb.save(output_filename)

    print("処理完了")

In [ ]:
# 総残業時間total_timeをファイルに書き込む
# 読み込むファイルは上のwrite_overtime_to_excelで出力したoutput_filenameに上書きする
update_overtime_request(output_filename, department_name, month, total_time)